# 실험 01 PN 접합 다이오드 IV 특성

*******************************************************************************
## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:7.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:7.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16):
    linestyle  = ['-', '-', '-', '-', '-', '-', '-', '-']
    colors     = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    markers    = ['o', 'v', 's', 'p', '*', 'h', 'x', 'd']

    markeverys = []
    x_step = (xs.max() - xs.min()) / marker_num
    for idx in range(len(markers)):
        x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
        marker_list = []
        for j in range(marker_num):
            tmp_array = np.where(xs > (x_start + (x_step * j)))
            if (len(tmp_array[0]) > 0):
                marker_list.append(tmp_array[0][0])
        markeverys.append(marker_list)

    plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=markers[style_idx], markevery=markeverys[style_idx], label=label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 결과 분석 (시험 과정 4)

### 회로도

* 사용한 소자
    + resistor: 100 Ohm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit.jpeg)

### 1. 실험을 통해 다이오드, D1에 흐르는 전류값을 구한다.

다이오드, D1에 다음 전압이 걸릴 때 저항, R1에 걸리는 전압을 측정한다.
* 0, 100, 200, 300, 400, 500, 550, 600, 650, 700 mV

실험 장비 해상도의 한계로 정확한 값의 전압을 걸 수 없을 경우 근사값 걸고 그 값을 기록한다.

In [ ]:
V_D_mV = [  0.000, 108.000, 200.000, 302.000, 405.000, 504.000, 552.000, 603.000, 651.000,  701.000]
V_R_mV = [  0.000,   0.000,   0.000,   0.000,   0.000,  10.000,  50.000, 140.000, 400.000, 1180.000]

저항 R1의 크기와, 걸리는 전압값을 사용해 다이오드, D1에 흐르는 전류값을 계산한다.

In [ ]:
R1_ohm = 100

V_D_mV = np.array(V_D_mV)
V_R_mV = np.array(V_R_mV)

I_D_mA = V_R_mV / R1_ohm
print_array('I_D_mA', I_D_mA)

### 2. 실험 결과에서 다이오드, D1에 걸리는 전압과 전류를 가져온다.

In [ ]:
vds = V_D_mV
ids = I_D_mA

### 3. 다이오드의 I<sub>D</sub>-V<sub>D</sub> 특성 곡선을 그린다.

다이오드, D1에 걸리는 전압을 x축, 전류를 y축으로 하는 그래프를 그린다.

이 그래프가 다이오드의 I<sub>D</sub>-V<sub>D</sub> 특성 곡선이다.

In [ ]:
plt.xlabel('V_D_mV')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.plot(vds, ids, 'b.-')
plt.show()

### 4. 다이오드, D1의 cut-in 전압 근사값을 구한다.

다이오드, D1에 걸리는 전압이 699 mV 지점에 접선을 그린다.

In [ ]:
fx = lambda x: np.interp(x, vds, ids)

tf = tangent_line(fx, 699)
tx = np.arange(600.0, 800.0, 10)
ty = tf(tx)

plt.xlabel('V_D_mV')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.plot(vds, ids, 'b.-')
plt.plot(tx, ty, 'g-')
plt.show()

접선과 x축이 만나는 지점의 x 값을 구한다.

이 값이 다이오드, D1의 cut-in 전압 근사값이다.

In [ ]:
V_cut_in_mV = np.interp(0.0, ty, tx)
print_value("V_cut_in_mV", V_cut_in_mV)